In [1]:
import decisionanalyzer as dan
from decisionanalyzer import *

For the code to work, the input dataset must be in the form of the example given:
- Dataframe pandas with features.
- A column for the binary membership classes, named "activity"
- the molecular SMILES as a string
- The last column is not strictly necessary.

In [2]:
#imports the training and validation set already divided and with the descriptors already calculated 
train_dtf = pd.read_csv("./PCA_descr/train_final_UV.csv", index_col=0)
test_dtf = pd.read_csv("./PCA_descr/test_final_UV.csv", index_col=0)
train_dtf

,T(N..N),T(N..O),T(N..S),T(N..P),T(N..F),T(N..Cl),T(N..Br),T(N..I),T(O..O),T(O..S),...,F10(O-Cl),F10(O-X),F10(S-F),F10(P-F),F10(P-X),F10(F-F),F10(F-X),activity,SMILES,UV
966,12,0,0,0,272,0,0,0,0,0,...,0,0,0,0,0,0,0,1,FC1=C(F)C(F)=C(C(F)=C1F)C1=C2C=CC3=[N]2[Pt++]2...,541
554,92,0,138,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,CN(C)C1=CC=C(C=C1)C1=CC(C2=CC=C(C=C2)C2=CC=CC3...,693
2122,18,126,60,0,0,0,0,0,150,228,...,0,0,0,0,0,0,0,1,[H][C@]1(CCCN1C1=[S][Ru+3]234([S-]1)[S-]C(=[S]...,575
192,54,0,0,0,156,44,0,0,0,0,...,0,0,0,0,0,0,0,1,C1(F)=C2[C-]3=C(C4=CC(C(C)(C)C)=CC=[N]4[Ir+3]3...,520
679,88,0,0,0,288,0,0,0,0,0,...,0,0,0,0,0,9,0,1,FC(F)(F)C1=NN2C(=C1)C1=[N](C=NC=C1)[Pt]21N2N=C...,501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778,0,0,2,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,C[S]1CC2=CC=CC=[N]2[Pt++]1([Cl-])[Cl-],291
3235,0,0,0,0,0,0,0,0,760,0,...,0,0,0,0,0,0,0,0,O[P@@]1([O-])O[P@](O)([O-])[Pt+3]234([Br-])[P@...,370
3270,20,31,0,0,0,0,0,0,6,0,...,0,0,0,0,0,0,0,0,CC(C)C1=CC=CC(C(C)C)=C1N1C2=CC=CC=C2[N]2=C1C1=...,378
3434,26,108,0,56,0,0,0,0,116,0,...,0,0,0,0,0,0,0,0,CCCC[P](CCCC)(CCCC)[Pt++]([C-]#CC1=CC=C(C=C1)C...,395


The models used in this case are XGBClassifier and DecisionTreeClassifier. The hyperparameters set were chosen by Grid Search.
Further details can be found in the reference article.

In [3]:
model_XGB = XGBClassifier(random_state=42,n_estimators= 400,max_depth=8, learning_rate=0.31)
model_DTC = DecisionTreeClassifier(random_state=42,max_depth=20,
                                   min_samples_split=2,min_samples_leaf=1,
                                   class_weight='balanced')
model_DTC_inst = [('DecisionTreeClassifier', model_DTC)]
model_XGB_inst = [('XGBClassifier', model_XGB)]

Further details on how the features importances were determined can be found in the referenced article.

In [4]:
feature_importance_df_sorted = pd.read_csv("./PCA_descr/feat_importance_final.csv",index_col=0)
feature_importance_df_sorted

,Feature,Importance
39,B01(C-X),0.031573
116,F02(C-N),0.011746
156,F03(C-X),0.010668
123,F02(C-X),0.010237
93,F01(C-C),0.008190
...,...,...
249,F06(C-X),-0.003341
77,B07(C-X),-0.003341
51,B02(N-X),-0.003341
129,F02(N-Cl),-0.003556


Running this code determines the evaluation metrics for both models, on the training set, with SkFoldCV (K = 5, 10, 15). The number of most important Features to be used for both models was determined with Recursive Features Elimination, as explained in the reference article.

In [5]:
#benchmark on the training set with SkFoldCV and k-values = 5, 10, 15
#or XGBC (49 features) and DTC (26 features)

kfold_results = []

k_values = [5, 10, 15]

for model_name, model_inst, top_features in [
    ("XGBClassifier", model_XGB_inst, feature_importance_df_sorted.Feature.head(49)),
    ("DecisionTreeClassifier", model_DTC_inst, feature_importance_df_sorted.Feature.head(26))
]:
    for k in k_values:
        fold_results = train_models_classification(model_inst, train_dtf[top_features], train_dtf.activity, kfold_splits=k)
        fold_results['Model'] = model_name
        fold_results['k'] = k

        kfold_results.append(fold_results)

kfold_results_df = pd.concat(kfold_results, ignore_index=True)

cols = ['Model', 'k'] + [col for col in kfold_results_df.columns if col not in ['Model', 'k']]
kfold_results_df = kfold_results_df[cols]

kfold_results_df.set_index(['Model', 'k'], inplace=True)

kfold_results_df = kfold_results_df.round(2)
kfold_results_df

Accuracy  Sensitivity  Specificity  Precision    F1
Model                  k                                                      
XGBClassifier          5       0.91         0.90         0.91       0.90  0.90
                       10      0.92         0.90         0.93       0.92  0.91
                       15      0.91         0.90         0.92       0.91  0.91
DecisionTreeClassifier 5       0.84         0.84         0.84       0.82  0.83
                       10      0.85         0.84         0.85       0.83  0.84
                       15      0.85         0.86         0.85       0.83  0.84

In [6]:
train_dtf_activity = train_dtf.copy()
train_dtf_activity = train_dtf_activity.reset_index(drop=True)

In [7]:
model = model_DTC
train_data = train_dtf_activity.iloc[:,:-2]
smiles_list =  train_dtf_activity.SMILES

#this is valid only for decision tree model
#model = trained model
#train_data = dataframe with all features and activity column
#smiles_list = list of smiles from dataset
#feature_importance_df_sorted = dataframe with features sorted by descending importance

helper = DecisionTreeHelper(model, train_data, smiles_list, feature_importance_df_sorted)

#run this to generate data for further analysis

helper.decision_paths(show_leafs=True)

#this code generates a dataframe indicating in which leaf node each molecule ended up

leaves_df = helper.find_leaves(return_list=True)

# this code allows you to get the list of "pure" nodes.
# "Pure nodes" means nodes that contain only molecules that are
# either all inactive or all active.
# Also, the directly connected node (via the parent node) will contain only molecules of the opposite class.
# In other words, all listed parent nodes split into two leaf nodes that contain only molecules
# that share the same class. This assures us that a node's division criterion is one
# that determines the division between active and inactive molecules.

pure_nodes = helper.find_splits(pure_node = True)

#leaf_descr gives more detailed information including the number of molecules in each leaf

leaf_descr = helper.describe(pure_nodes)

Node 692 is connected to node 693 by the common node 691
Node 692 has 3 active compound(s)
Node 691 has 1 inactive compound(s)
----------
Node 700 is connected to node 701 by the common node 699
Node 700 has 2 active compound(s)
Node 699 has 1 inactive compound(s)
----------
Node 703 is connected to node 704 by the common node 702
Node 703 has 1 active compound(s)
Node 702 has 1 inactive compound(s)
----------
Node 713 is connected to node 714 by the common node 712
Node 713 has 2 active compound(s)
Node 712 has 3 inactive compound(s)
----------
Node 719 is connected to node 720 by the common node 718
Node 719 has 1 active compound(s)
Node 718 has 8 inactive compound(s)
----------
Node 736 is connected to node 737 by the common node 735
Node 736 has 1 active compound(s)
Node 735 has 1 inactive compound(s)
----------
Node 781 is connected to node 782 by the common node 780
Node 781 has 15 active compound(s)
Node 780 has 1 inactive compound(s)
----------
Node 679 is connected to node 680

Node 472 is connected to node 473 by the common node 471
Node 472 has 2 active compound(s)
Node 471 has 7 inactive compound(s)
----------
Node 476 is connected to node 477 by the common node 475
Node 476 has 1 active compound(s)
Node 475 has 6 inactive compound(s)
----------
Node 484 is connected to node 485 by the common node 483
Node 484 has 1 active compound(s)
Node 483 has 1 inactive compound(s)
----------
Node 506 is connected to node 507 by the common node 505
Node 506 has 1 active compound(s)
Node 505 has 3 inactive compound(s)
----------
Node 413 is connected to node 414 by the common node 412
Node 413 has 1 active compound(s)
Node 412 has 1 inactive compound(s)
----------
Node 398 is connected to node 399 by the common node 397
Node 398 has 1 active compound(s)
Node 397 has 1 inactive compound(s)
----------
Node 292 is connected to node 293 by the common node 291
Node 292 has 1 active compound(s)
Node 291 has 1 inactive compound(s)
----------
Node 299 is connected to node 300 

In [8]:
#This code is useful if you are interested in which node a particular feature corresponds to.
#Just enter the feature name in string form as the function argument.
#If you do not enter anything, the most important feature according to the model is automatically taken.
helper.find_node_by_feat()

Descriptor 'B01(C-X)' is the decision criteria of node(s): [780]


In [9]:
leaves_df

,sample,leaf
0,FC1=C(F)C(F)=C(C(F)=C1F)C1=C2C=CC3=[N]2[Pt++]2...,784
1,CN(C)C1=CC=C(C=C1)C1=CC(C2=CC=C(C=C2)C2=CC=CC3...,784
2,[H][C@]1(CCCN1C1=[S][Ru+3]234([S-]1)[S-]C(=[S]...,744
3,C1(F)=C2[C-]3=C(C4=CC(C(C)(C)C)=CC=[N]4[Ir+3]3...,1035
4,FC(F)(F)C1=NN2C(=C1)C1=[N](C=NC=C1)[Pt]21N2N=C...,182
...,...,...
3707,C[S]1CC2=CC=CC=[N]2[Pt++]1([Cl-])[Cl-],10
3708,O[P@@]1([O-])O[P@](O)([O-])[Pt+3]234([Br-])[P@...,10
3709,CC(C)C1=CC=CC(C(C)C)=C1N1C2=CC=CC=C2[N]2=C1C1=...,990
3710,CCCC[P](CCCC)(CCCC)[Pt++]([C-]#CC1=CC=C(C=C1)C...,841


In [10]:
pure_nodes

,left_leaves,parent_leaves,right_leaves
0,10,9,11
1,16,15,17
2,20,19,21
3,23,22,24
4,27,26,28
...,...,...,...
173,1047,1046,1048
174,1050,1049,1051
175,1059,1058,1060
176,1065,1064,1066


In [11]:
leaf_descr.sort_values(by='leaf_count',ascending=False)

,leaf,leaf_conn,leaf_common,leaf_count,leaf_value,leaf_conn_count,leaf_conn_value
18,955,956,954,18,1,1,0
41,159,160,158,17,1,1,0
6,781,782,780,15,1,1,0
13,611,612,610,13,1,1,0
35,927,928,926,11,1,1,0
...,...,...,...,...,...,...,...
47,127,128,126,1,1,1,0
24,1059,1060,1058,1,1,1,0
5,736,737,735,1,1,1,0
53,105,106,104,1,1,1,0


In [12]:
# Given the ID number of a node, this code provides all the nodes from which it is derived backward.
# In addition, for each node found, information regarding its division is returned.
helper.backward_search(782)

The nodes from which node 782 is derived are, backwards: [780, 779, 778, 774, 773, 772, 771, 765, 764, 763, 729, 728, 0]
Below is the list of nodes resulting from the split of the listed nodes
Node 780 is splitted in: left_child = 781, right_child = 782
Node 779 is splitted in: left_child = 780, right_child = 783
Node 778 is splitted in: left_child = 779, right_child = 788
Node 774 is splitted in: left_child = 775, right_child = 778
Node 773 is splitted in: left_child = 774, right_child = 791
Node 772 is splitted in: left_child = 773, right_child = 796
Node 771 is splitted in: left_child = 772, right_child = 807
Node 765 is splitted in: left_child = 766, right_child = 771
Node 764 is splitted in: left_child = 765, right_child = 808
Node 763 is splitted in: left_child = 764, right_child = 811
Node 729 is splitted in: left_child = 730, right_child = 763
Node 728 is splitted in: left_child = 729, right_child = 828
Node 0 is splitted in: left_child = 1, right_child = 728


It is observed from the above results that the most important feature is in node 780.
This node divides into leaves 781 and 782.
Leaf 781 contains 15 active compounds, leaf 782 contains one inactive compound.

In [13]:
leaves_df[leaves_df["leaf"] == 782]

,sample,leaf
1770,CCCCN1C2=CC3=CC=CC=C3C=C2[N]2=C1C1=CC(=CC3=[C-...,782


The inactive compound, number 1770, follows these decisions within the tree.
It can be seen at the end how the last final decision is precisely that of the most important feature B01(C-X) = 1 > 0.5
Molecule 1770 is then selected along with the tree classification rules as a starting point for a design.
More details in the reference article.

In [14]:
helper.sample_decision_path(1770)

Rules followed by sample number 1770:

Decision node 0: F05(C-X) = 7 > 6.5
Decision node 728: F01(C-X) = 1 <= 1.5
Decision node 729: F03(C-N) = 24 > 7.0
Decision node 763: F01(N-N) = 0 <= 1.5
Decision node 764: F01(C-C) = 40 <= 112.0
Decision node 765: F02(C-X) = 6 > 4.5
Decision node 771: F03(C-X) = 6 <= 47.0
Decision node 772: F07(C-P) = 0 <= 3.5
Decision node 773: F01(C-C) = 40 <= 78.5
Decision node 774: F02(C-N) = 16 > 2.5
Decision node 778: T(O..O) = 0 <= 1004.0
Decision node 779: F03(C-X) = 6 <= 6.5
Decision node 780: B01(C-X) = 1 > 0.5
------------------------
Sample 1770 ends up in node 782
and it is classified as inactive
------------------------
